In [1]:
import os
import random
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import auc, roc_auc_score

from lightgbm import LGBMClassifier
import lightgbm as lgb

from hyperopt import hp, tpe, Trials, fmin


In [2]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
submission = pd.read_csv("../input/sample_submission.csv")

In [3]:
train.shape, test.shape

((200000, 202), (200000, 201))

In [4]:
train.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [5]:
np.sum(train['target']) / len(train['target'])

0.10049

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
                                                        train.drop(['ID_code', 'target'], axis = 1),
                                                        train['target'],
                                                        test_size = 0.15,
                                                        random_state = 42,
                                                        stratify = train['target']
                                                    )

In [7]:
np.sum(y_train) / len(y_train), np.sum(y_test) / len(y_test)

(0.10048823529411764, 0.1005)

In [8]:
def objective(params):
    
    n_folds = 3
    params = {
                'n_estimators': int(params['n_estimators']),
                'num_leaves': int(params['num_leaves']),
                'learning_rate': float(params['learning_rate']),
                'subsample_for_bin': int(params['subsample_for_bin']),
                'min_child_samples': int(params['min_child_samples']),
                'reg_alpha': float(params['reg_alpha']),
                'reg_lambda': float(params['reg_lambda'])
             }
    
    clf = LGBMClassifier(**params)
    score1 = cross_val_score(clf, X_train, y_train, scoring = 'roc_auc', cv = StratifiedKFold(n_splits = n_folds)).mean()
    clf.fit(
                X_train, y_train,
                eval_set = [(X_test, y_test)],
                eval_metric = 'auc',
                early_stopping_rounds = 3,
                verbose = False,
            )
    score2 = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
    score = 2 / ((1 / score1) + (1 / score2))

    return 1 - score

In [9]:
space = {
            'n_estimators': hp.quniform('n_estimators', 50, 1500, 25),
            'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
            'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
            'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
            'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
            'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
            'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0)
        }
tpe_algo = tpe.suggest
tpe_trials = Trials()

# tpe_best = fmin(fn = objective, space = space, algo = tpe_algo, trials = tpe_trials, 
#                 max_evals = 15, rstate = np.random.RandomState(42))

In [10]:
# tpe_best

In [11]:
# params = {
#             'n_estimators': int(tpe_best['n_estimators']),
#             'num_leaves': int(tpe_best['num_leaves']),
#             'learning_rate': float(tpe_best['learning_rate']),
#             'subsample_for_bin': int(tpe_best['subsample_for_bin']),
#             'min_child_samples': int(tpe_best['min_child_samples']),
#             'reg_alpha': float(tpe_best['reg_alpha']),
#             'reg_lambda': float(tpe_best['reg_lambda'])
#          }

In [12]:
params = {
            'bagging_freq': 5,
            'bagging_fraction': 0.331,
            'boost_from_average':'false',
            'boost': 'gbdt',
            'feature_fraction': 0.0405,
            'learning_rate': 0.0083,
            'max_depth': -1,
            'metric':'auc',
            'min_data_in_leaf': 80,
            'min_sum_hessian_in_leaf': 10.0,
            'num_leaves': 13,
            'num_threads': 8,
            'tree_learner': 'serial',
            'objective': 'binary',
            'verbosity': 1
        }

params = {
             'num_leaves': 8,
             'min_data_in_leaf': 42,
             'objective': 'binary',
             'max_depth': 16,
             'learning_rate': 0.0123,
             'boosting': 'gbdt',
             'bagging_freq': 5,
             'feature_fraction': 0.8201,
             'bagging_seed': 11,
             'reg_alpha': 1.728910519108444,
             'reg_lambda': 4.9847051755586085,
             'random_state': 42,
             'metric': 'auc',
             'verbosity': -1,
             'subsample': 0.81,
             'min_gain_to_split': 0.01077313523861969,
             'min_child_weight': 19.428902804238373,
             'num_threads': 4
         }

lgb_base = LGBMClassifier(**params, n_estimators = 20000)

lgb_base.fit(
                X_train, y_train,
                eval_set = [(X_test, y_test)],
                eval_metric = 'auc',
                early_stopping_rounds = 200,
                verbose = 1000,
            )

Training until validation scores don't improve for 200 rounds.
[1000]	valid_0's auc: 0.857988
[2000]	valid_0's auc: 0.879314
[3000]	valid_0's auc: 0.88762
[4000]	valid_0's auc: 0.892024
[5000]	valid_0's auc: 0.894185
[6000]	valid_0's auc: 0.895363
[7000]	valid_0's auc: 0.896013
[8000]	valid_0's auc: 0.896308
[9000]	valid_0's auc: 0.896479
Early stopping, best iteration is:
[8891]	valid_0's auc: 0.896513


LGBMClassifier(bagging_freq=5, bagging_seed=11, boosting='gbdt',
        boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        feature_fraction=0.8201, importance_type='split',
        learning_rate=0.0123, max_depth=16, metric='auc',
        min_child_samples=20, min_child_weight=19.428902804238373,
        min_data_in_leaf=42, min_gain_to_split=0.01077313523861969,
        min_split_gain=0.0, n_estimators=20000, n_jobs=-1, num_leaves=8,
        num_threads=4, objective='binary', random_state=42,
        reg_alpha=1.728910519108444, reg_lambda=4.9847051755586085,
        silent=True, subsample=0.81, subsample_for_bin=200000,
        subsample_freq=0, verbosity=-1)

In [13]:
print("Training AUC:")
print(roc_auc_score(y_train, lgb_base.predict_proba(X_train)[:, 1]))
print("\n")
print("Testing AUC:")
print(roc_auc_score(y_test, lgb_base.predict_proba(X_test)[:, 1]))

Training AUC:
0.9461715902596058


Testing AUC:
0.89651303730867


In [14]:
submission['target'] = 1 - lgb_base.predict_proba(test.drop('ID_code', axis = 1))

In [15]:
submission.head()

,ID_code,target
0,test_0,0.091183
1,test_1,0.216087
2,test_2,0.205930
3,test_3,0.169996
4,test_4,0.049044


In [16]:
submission.to_csv("submission_hyperparams_borrowed.csv", index = False)